In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Flatten
from sklearn.svm import SVC

ERROR! Session/line number was not unique in database. History logging moved to new session 4


In [2]:
##selected classses
classes = ["Lemon", "Raspberry", "Mandarine", "Pear 2", "Apple Red 1", "Strawberry", "Pomegranate", "Pepino", "Kaki", "Apricot", "Ginger Root", "Huckleberry"]

In [3]:
TRAIN_DIR = "../data/Training"
TEST_DIR = "../data/Test"

In [4]:
## our sweet NN
base_model = VGG16(weights='imagenet', include_top = False , input_shape=(224,224, 3))
print(base_model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
for layer in base_model.layers:
    layer.trainable = False

In [6]:
# max_pooling = MaxPool2D()()
flatten_svm= Flatten()(base_model.layers[-1].output)
# add new classifier layers

# class_dense_svm = Dense(4096, activation='relu')(base_model.layers[-1].output) ##maybe not relu...
features_svm = Model(inputs=base_model.input, outputs=flatten_svm)
features_svm.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# TEST Z MNIEJSZĄ ILOŚCIĄ DANYCH

In [7]:
small_trdata_gen = ImageDataGenerator(rescale=1./255)
smalltrain = small_trdata_gen.flow_from_directory(directory=TRAIN_DIR, target_size=(224,224), batch_size=1, color_mode="rgb", classes=classes, class_mode="categorical", shuffle=False)

# validation subset split LOOOOOOOOOOL
# smallval = small_trdata_gen.flow_from_directory(directory=TRAIN_DIR, subset='validation', target_size=(224,224), batch_size=1, color_mode="rgb", classes=classes, class_mode="categorical", shuffle=False)
# useless for now

Found 11524 images belonging to 12 classes.


In [8]:
small_tsdata_gen = ImageDataGenerator(rescale=1./255)
smalltest = small_tsdata_gen.flow_from_directory(directory=TEST_DIR, target_size=(224,224),batch_size=1, color_mode="rgb", classes=classes,class_mode="categorical", shuffle=False)

Found 1981 images belonging to 12 classes.


In [9]:
small_train_features = features_svm.predict(smalltrain, verbose=1)
small_test_features = features_svm.predict(smalltest, verbose=1)

1981/1981 [==============================] - 381s 193ms/step


In [10]:
# FOR SMALLER SETS
small_X_train = small_train_features
small_y_train = np.array(smalltrain.labels)
small_X_test = small_test_features
small_y_test = np.array(smalltest.labels)

In [11]:
import sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_val, y_train, y_val = train_test_split(small_X_train, small_y_train, test_size=0.2, random_state=85)

# RBF

In [13]:
svm_rbf = svm.SVC(kernel='rbf', C=100, verbose=2) #, probability=True, random_state=69)
svm_rbf.fit(X_train, y_train)
y_pred_train_rbf = svm_rbf.predict(X_train)
y_pred_test_rbf = svm_rbf.predict(small_X_test)

[LibSVM]

In [14]:
rbf_report_train = sklearn.metrics.classification_report(y_train, y_pred_train_rbf)
rbf_report_test = sklearn.metrics.classification_report(small_y_test, y_pred_test_rbf)

In [15]:
print(rbf_report_train)
print(rbf_report_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       772
           1       1.00      1.00      1.00       769
           2       1.00      1.00      1.00       761
           3       1.00      1.00      1.00       774
           4       1.00      1.00      1.00       770
           5       1.00      1.00      1.00       762
           6       1.00      1.00      1.00       784
           7       1.00      1.00      1.00       773
           8       1.00      1.00      1.00       774
           9       1.00      1.00      1.00       771
          10       1.00      1.00      1.00       760
          11       1.00      1.00      1.00       749

    accuracy                           1.00      9219
   macro avg       1.00      1.00      1.00      9219
weighted avg       1.00      1.00      1.00      9219

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       164
           1       1.00 

## k-fold

In [16]:
# from sklearn.metrics import classification_report, accuracy_score
# from sklearn.model_selection import cross_val_score, cross_val_predict

In [17]:
# cv_results = cross_val_score(svm_rbf, X_train, y_train, cv=10)

In [18]:
# print(cv_results)

In [19]:
# predicted = cross_val_predict(svm_rbf, X_train, y_train, cv=10)

In [20]:
# print(accuracy_score(y_train, predicted))

# Linear

In [21]:
svm_linear = svm.SVC(kernel='linear',C=1, verbose=2) #, probability=True, random_state=69)
svm_linear.fit(X_train, y_train)
y_pred_train_lin = svm_linear.predict(X_train)
y_pred_test_lin = svm_linear.predict(small_X_test)

[LibSVM]

In [22]:
lin_report_train = sklearn.metrics.classification_report(y_train, y_pred_train_lin)
lin_report_test = sklearn.metrics.classification_report(small_y_test, y_pred_test_lin)

In [24]:
print(lin_report_train)
print(lin_report_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       772
           1       1.00      1.00      1.00       769
           2       1.00      1.00      1.00       761
           3       1.00      1.00      1.00       774
           4       1.00      1.00      1.00       770
           5       1.00      1.00      1.00       762
           6       1.00      1.00      1.00       784
           7       1.00      1.00      1.00       773
           8       1.00      1.00      1.00       774
           9       1.00      1.00      1.00       771
          10       1.00      1.00      1.00       760
          11       1.00      1.00      1.00       749

    accuracy                           1.00      9219
   macro avg       1.00      1.00      1.00      9219
weighted avg       1.00      1.00      1.00      9219

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       164
           1       1.00 

# Poly

In [25]:
svm_poly = svm.SVC(kernel='poly',C=1, degree=2, verbose=2) #, probability=True, random_state=69)
svm_poly.fit(X_train, y_train)
y_pred_train_poly = svm_poly.predict(X_train)
y_pred_test_poly = svm_poly.predict(small_X_test)

[LibSVM]

In [26]:
poly_report_train = sklearn.metrics.classification_report(y_train, y_pred_train_poly)
poly_report_test = sklearn.metrics.classification_report(small_y_test, y_pred_test_poly)

In [27]:
print(poly_report_train)
print(poly_report_test)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       772
           1       1.00      1.00      1.00       769
           2       1.00      1.00      1.00       761
           3       1.00      1.00      1.00       774
           4       1.00      1.00      1.00       770
           5       1.00      1.00      1.00       762
           6       1.00      1.00      1.00       784
           7       1.00      1.00      1.00       773
           8       1.00      1.00      1.00       774
           9       0.98      1.00      0.99       771
          10       1.00      1.00      1.00       760
          11       1.00      1.00      1.00       749

    accuracy                           1.00      9219
   macro avg       1.00      1.00      1.00      9219
weighted avg       1.00      1.00      1.00      9219

              precision    recall  f1-score   support

           0       0.89      1.00      0.94       164
           1       1.00 

# Save results

In [30]:
from joblib import dump, load

# RBF data

In [31]:
dump(rbf_report_train, './results/rbf_report_train_C_1.joblib')
dump(rbf_report_test, './results/rbf_report_test_C_1.joblib')
dump(svm_rbf, './models/svm_rbf_C_1.joblib')

['./models/svm_rbf_C_1.joblib']

# Linear data

In [32]:
dump(lin_report_train, './results/lin_report_train_C_1.joblib')
dump(lin_report_test, './results/lin_report_test_C_1.joblib')
dump(svm_linear, './models/svm_linear_C_1.joblib')

['./models/svm_linear_C_1.joblib']

# Polynomial data

In [33]:
dump(poly_report_train, './results/poly_report_train_C_1.joblib')
dump(poly_report_test, './results/poly_report_test_C_1.joblib')
dump(svm_poly, './models/svm_poly_C_1.joblib')

['./models/svm_poly_C_1.joblib']